> We have already established that using annotated HeLa m6A sites, we can observe changes in genes with m6A sites (HL-60) cells. In order to confirm this m6A sites, we performed MeRIP-seq in treated and untreated cells, and did observe a general increase in m6A levels upon treatments for a large number of annotated sites. Here, our goal is to indpendently analyze the MeRIP data without relying on HeLa annotations and use it to define a **treatment-induced hyper-methylation sites**. We will then assess the location and behaviour of these targets across the other datasets generated in this study.

# meRIP-seq data 

### Meta-gene plot

As a quality control, we confirm mRNA Methylation sites enriched in 3′ UTRs and near stop codons same as this Cell paper 
https://doi.org/10.1016/j.cell.2012.05.003

<img src="meRIP-seq/plots/Guitar_mrna_test.png" title="DRACH and RGAC" style="width:600px">

### Motif analysis 

<img src="meRIP-seq/fire/motifs_of_interest_FIRE_summary.png" title="DRACH and RGAC" style="width:1000px">


__Results from FIRE__ shown above indicate that the known m6A site **DRACH** and **RGAC** (or `[AG]GAC` and `[AGT][AG]AC[ACT]` in regular expression form) is significantly enriched among the **~4700** methylations sites.

### Test enrichment of treatment-induced hyper/hypo-methylation sites
[TEISER](https://github.com/goodarzilab/TEISER)
Here, I aim to identify the genes that are hyper or hypo methylated as genesets, and see if they have enriched accross all genes; the input table is list of genes with control vs. treated fold change of RNA expression, RNA stability and translational efficiency. 

Prepare inputs

In [4]:
%%bash
mkdir -p combined_analysis/

### hyper_methylation gene list
# subset by threshold - logFC >= 2
awk '$2>=2{print $1}' meRIP-seq/d_mtyl_T_vs_U.txt | \
awk '!seen[$1]++' > combined_analysis/hyper_mtyl.txt

### hypo_methylation gene list
# subset by threshold - logFC < 2
awk '$2<-2 {print $1}' meRIP-seq/d_mtyl_T_vs_U.txt | \
awk '!seen[$1]++' > combined_analysis/hypo_mtyl.txt

In [5]:
ls combined_analysis/

hyper_mtyl.txt  hypo_mtyl.txt


In [7]:
ls RNA-seq/hl60-stbl/*delta_stbl.txt

RNA-seq/hl60-stbl/120h_delta_stbl.txt  RNA-seq/hl60-stbl/6h_delta_stbl.txt


In [6]:
ls RNA-seq/other-stbl/*delta_stbl.txt

RNA-seq/other-stbl/kg1_delta_stbl.txt
RNA-seq/other-stbl/molm14_delta_stbl.txt
RNA-seq/other-stbl/ociaml2_delta_stbl.txt
RNA-seq/other-stbl/ociaml3_delta_stbl.txt
RNA-seq/other-stbl/thp1_delta_stbl.txt


In [16]:
ls Ribo-seq/Results/

Decitabine.Rmd              lnTE_T_vs_U/    README.md
fit1_deci_fdr_qval_g_0.csv  make_readme.sh  report.ipynb
fit1_deci_fdr_qval_g.csv    plots/


In [10]:
!source ~/.bashrc

In [10]:
%%bash

declare -a Experiments=(
# Ribo-seq
'Ribo-seq/lnTE_T_vs_U.txt' 

## HL-60 RNA-seq 
# RNA experssion 
'RNA-seq/hl60-exp/6h_delta_exp.txt' 'RNA-seq/hl60-exp/72h_delta_exp.txt' 'RNA-seq/hl60-exp/120h_delta_exp.txt' 
# RNA stability  
'RNA-seq/hl60-stbl/120h_delta_stbl.txt'  'RNA-seq/hl60-stbl/6h_delta_stbl.txt'

## 5 other AML cell lines RNA-seq
# RNA experssion 
'RNA-seq/other-exp/kg1_delta_exp.txt' 'RNA-seq/other-exp/molm14_delta_exp.txt'
'RNA-seq/other-exp/ociaml2_delta_exp.txt' 'RNA-seq/other-exp/ociaml3_delta_exp.txt'
'RNA-seq/other-exp/thp1_delta_exp.txt'
# RNA stability  
'RNA-seq/other-stbl/kg1_delta_stbl.txt' 'RNA-seq/other-stbl/molm14_delta_stbl.txt' 
'RNA-seq/other-stbl/ociaml2_delta_stbl.txt' 'RNA-seq/other-stbl/ociaml3_delta_stbl.txt'
'RNA-seq/other-stbl/thp1_delta_stbl.txt'
)

for exp in "${Experiments[@]}"; do
    echo $exp
    base=`basename $exp`
    base=${base/.txt/}
    # get intersect 
    awk 'NR==FNR{A[$1];next}$1 in A' $exp combined_analysis/hyper_mtyl.txt > combined_analysis/hyper_mtyl_${base}.txt

    echo hyper_methylation
    perl $TEISERDIR/run_mi_gene_list.pl \
        --expfile=$exp \
        --genefile=combined_analysis/hyper_mtyl_${base}.txt \
        --exptype=continuous \
        --ebins=7 \
        --species=human \
        --doremovedups=0 \
        --doremoveextra=0
    # remove results from previous run 
    rm -vr combined_analysis/hyper_mtyl_${base}_GENESET
    mv -v ${exp}_GENESET combined_analysis/hyper_mtyl_${base}_GENESET
        
    awk 'NR==FNR{A[$1];next}$1 in A' $exp combined_analysis/hypo_mtyl.txt > combined_analysis/hypo_mtyl_${base}.txt
    
    echo hypo_methylation
    perl $TEISERDIR/run_mi_gene_list.pl \
        --expfile=$exp \
        --genefile=combined_analysis/hypo_mtyl_${base}.txt \
        --exptype=continuous \
        --ebins=7 \
        --species=human \
        --doremovedups=0 \
        --doremoveextra=0
    # remove results from previous run 
    rm -vr combined_analysis/hypo_mtyl_${base}_GENESET
    mv -v ${exp}_GENESET combined_analysis/hypo_mtyl_${base}_GENESET
done

Ribo-seq/lnTE_T_vs_U.txt
hyper_methylation
The TEISERDIR environment variable is /rumi/shams/abe/TEISERv1.1
Thu Dec 10 16:17:01 UTC 2020
Remove duplicates, create Ribo-seq/lnTE_T_vs_U.txt_GENESET/lnTE_T_vs_U.txt
Quantizing the input file.
step 2: seed optimization.
Expfile loaded: 6309 values...
Adding small values...
Quantizing the input vector...Done
Number of clusters: 7
calculating the p-value matrix.
Allocating memory ... Done
doing stats: mi = 0.001502
pass = 0.041900
z = 2.126055
freq 0: 0.091010	82	901	596	6309
freq 1: 0.081021	73	901	596	6309
freq 2: 0.085461	77	901	596	6309
freq 3: 0.086570	78	901	596	6309
freq 4: 0.086570	78	901	596	6309
freq 5: 0.112098	101	901	596	6309
freq 6: 0.118494	107	903	596	6309
step 7: drawing matrix.
Reading MI data ... Done.
Start drawing
0.76	-0.4
0
Outputing EPS file Ribo-seq/lnTE_T_vs_U.txt_GENESET/lnTE_T_vs_U.txt.summary.eps
Convert to PDF Ribo-seq/lnTE_T_vs_U.txt_GENESET/lnTE_T_vs_U.txt.summary.pdf
ps2pdf -dEPSCrop -dAutoRotatePages=/None Ri

Option suffix requires an argument
Option suffix requires an argument
Option suffix requires an argument
rm: cannot remove 'combined_analysis/hyper_mtyl_6h_delta_exp_GENESET': No such file or directory
Option suffix requires an argument
Option suffix requires an argument
rm: cannot remove 'combined_analysis/hyper_mtyl_72h_delta_exp_GENESET': No such file or directory
Option suffix requires an argument
Option suffix requires an argument
rm: cannot remove 'combined_analysis/hyper_mtyl_120h_delta_exp_GENESET': No such file or directory
Option suffix requires an argument
Option suffix requires an argument
rm: cannot remove 'combined_analysis/hyper_mtyl_120h_delta_stbl_GENESET': No such file or directory
Option suffix requires an argument
Option suffix requires an argument
rm: cannot remove 'combined_analysis/hyper_mtyl_6h_delta_stbl_GENESET': No such file or directory
Option suffix requires an argument
Option suffix requires an argument
Option suffix requires an argument
Option suffix requ

In [11]:
%%bash 
for pdf in combined_analysis/*_GENESET/*.txt.summary.pdf; do 
    png=${pdf/.pdf/.png}
    di=`dirname $pdf`
    out=`basename $di`
    outpng=${out/_GENESET/.png}
    outpdf=${out/_GENESET/.pdf}
    
    bash /rumi/shams/abe/Workflows/my_scripts/pdf2png.sh $pdf
    cp -v $pdf combined_analysis/$outpdf
    mv -v $png combined_analysis/$outpng
done 

combined_analysis/hyper_mtyl_120h_delta_exp_GENESET/120h_delta_exp.txt.summary.pdf > combined_analysis/hyper_mtyl_120h_delta_exp_GENESET/120h_delta_exp.txt.summary.png
done!
'combined_analysis/hyper_mtyl_120h_delta_exp_GENESET/120h_delta_exp.txt.summary.pdf' -> 'combined_analysis/hyper_mtyl_120h_delta_exp.pdf'
renamed 'combined_analysis/hyper_mtyl_120h_delta_exp_GENESET/120h_delta_exp.txt.summary.png' -> 'combined_analysis/hyper_mtyl_120h_delta_exp.png'
combined_analysis/hyper_mtyl_120h_delta_stbl_GENESET/120h_delta_stbl.txt.summary.pdf > combined_analysis/hyper_mtyl_120h_delta_stbl_GENESET/120h_delta_stbl.txt.summary.png
done!
'combined_analysis/hyper_mtyl_120h_delta_stbl_GENESET/120h_delta_stbl.txt.summary.pdf' -> 'combined_analysis/hyper_mtyl_120h_delta_stbl.pdf'
renamed 'combined_analysis/hyper_mtyl_120h_delta_stbl_GENESET/120h_delta_stbl.txt.summary.png' -> 'combined_analysis/hyper_mtyl_120h_delta_stbl.png'
combined_analysis/hyper_mtyl_6h_delta_exp_GENESET/6h_delta_exp.txt.summary

In [12]:
%%bash 
touch combined_analysis/README.md
for f in combined_analysis/*.png; do 
    b=`basename $f`
    t=${b/.png/}
    echo '### '$t >> combined_analysis/README.md
    echo -e "<img src=\""$f"\" title=\""$t"\" style=\"width:1000px\">\n" >> combined_analysis/README.md
done 

# RNA Expression

I've used `salmon` for alignment and `DESeq2` for ANNOVA like analysis (see [DESeq2 for time-series-experiments](http://bioconductor.org/packages/devel/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#time-series-experiments.)). Mainly, the linear model designed as `~condition + time + condition:time` to include both time and treatment together in the model. 

## HL-60 cell line - Time series experiment

## 5 other AML cell lines 

# RNA Stability 

Separately, I've used `STAR` for alignment and then, I've used `featureCount`
to evalute intronic and exonic counts (similar to [CRIES](https://github.com/csglab/CRIES)). 
Then, [REMBRANDTS](https://github.com/csglab/REMBRANDTS) is the tool to estimate unbias transcript stability. Then, I use `limma` for differential analysis. Here we only check the log2FC of treated vs. control 

## HL-60 cell line - Time series experiment

Let's redraw heatmaps using `--min=-3 --max=3` thresholds. 

In [14]:
%%bash 
declare -a Genesets=('hyper_mtyl' 'hypo_mtyl')
declare -a Times=('6h' '120h')
for time in "${Times[@]}"; do
    for geneset in "${Genesets[@]}"; do
        echo $time $geneset    
        perl $TEISERDIR/Scripts/teiser_draw_matrix.pl \
        --pvmatrixfile=combined_analysis/${geneset}_${time}_delta_stbl_GENESET/${time}_delta_stbl.txt.matrix \
        --summaryfile=combined_analysis/${geneset}_${time}_delta_stbl_GENESET/${time}_delta_stbl.txt.summary \
        --expfile=combined_analysis/${geneset}_${time}_delta_stbl_GENESET/${time}_delta_stbl.txt \
        --quantized=0 \
        --colmap=$TEISERDIR/Scripts/HEATMAPS/cmap_1.txt \
        --order=0 \
        --min=-3 --max=3 \
        --cluster=5 
    done 
done 

6h hyper_mtyl
Reading MI data ... Done.
Start drawing
0.26	-0.32
0
Outputing EPS file combined_analysis/hyper_mtyl_6h_delta_stbl_GENESET/6h_delta_stbl.txt.summary.eps
Convert to PDF combined_analysis/hyper_mtyl_6h_delta_stbl_GENESET/6h_delta_stbl.txt.summary.pdf
ps2pdf -dEPSCrop -dAutoRotatePages=/None combined_analysis/hyper_mtyl_6h_delta_stbl_GENESET/6h_delta_stbl.txt.summary.eps combined_analysis/hyper_mtyl_6h_delta_stbl_GENESET/6h_delta_stbl.txt.summary.pdf
Finished.
6h hypo_mtyl
Reading MI data ... Done.
Start drawing
0.26	-0.32
0
Outputing EPS file combined_analysis/hypo_mtyl_6h_delta_stbl_GENESET/6h_delta_stbl.txt.summary.eps
Convert to PDF combined_analysis/hypo_mtyl_6h_delta_stbl_GENESET/6h_delta_stbl.txt.summary.pdf
ps2pdf -dEPSCrop -dAutoRotatePages=/None combined_analysis/hypo_mtyl_6h_delta_stbl_GENESET/6h_delta_stbl.txt.summary.eps combined_analysis/hypo_mtyl_6h_delta_stbl_GENESET/6h_delta_stbl.txt.summary.pdf
Finished.
120h hyper_mtyl
Reading MI data ... Done.
Start drawi

## 5 other AML cell lines 

In [19]:
%%bash 
declare -a Genesets=('hyper_mtyl' 'hypo_mtyl')
declare -a Lines=('kg1' 'ociaml2' 'molm14' 'ociaml3' 'thp1' 'ociaml2')

for geneset in "${Genesets[@]}"; do
    for line in "${Lines[@]}"; do
        echo $geneset
        perl $TEISERDIR/Scripts/teiser_draw_matrix.pl \
        --pvmatrixfile=combined_analysis/${geneset}_${line}_delta_stbl_GENESET/${line}_delta_stbl.txt.matrix \
        --summaryfile=combined_analysis/${geneset}_${line}_delta_stbl_GENESET/${line}_delta_stbl.txt.summary \
        --expfile=combined_analysis/${geneset}_${line}_delta_stbl_GENESET/${line}_delta_exp.txt \
        --quantized=0 \
        --colmap=$TEISERDIR/Scripts/HEATMAPS/cmap_1.txt \
        --order=0 \
        --min=-3 --max=3 \
        --cluster=5 
    done
done

hyper_mtyl
Reading MI data ... Done.
Start drawing
1.14	-1.42
0
Outputing EPS file combined_analysis/hyper_mtyl_kg1_delta_stbl_GENESET/kg1_delta_exp.txt.summary.eps
Convert to PDF combined_analysis/hyper_mtyl_kg1_delta_stbl_GENESET/kg1_delta_exp.txt.summary.pdf
ps2pdf -dEPSCrop -dAutoRotatePages=/None combined_analysis/hyper_mtyl_kg1_delta_stbl_GENESET/kg1_delta_exp.txt.summary.eps combined_analysis/hyper_mtyl_kg1_delta_stbl_GENESET/kg1_delta_exp.txt.summary.pdf
Finished.
hyper_mtyl
Reading MI data ... Done.
Start drawing
1.32	-1.52
0
Outputing EPS file combined_analysis/hyper_mtyl_ociaml2_delta_stbl_GENESET/ociaml2_delta_exp.txt.summary.eps
Convert to PDF combined_analysis/hyper_mtyl_ociaml2_delta_stbl_GENESET/ociaml2_delta_exp.txt.summary.pdf
ps2pdf -dEPSCrop -dAutoRotatePages=/None combined_analysis/hyper_mtyl_ociaml2_delta_stbl_GENESET/ociaml2_delta_exp.txt.summary.eps combined_analysis/hyper_mtyl_ociaml2_delta_stbl_GENESET/ociaml2_delta_exp.txt.summary.pdf
Finished.
hyper_mtyl
Rea

In [23]:
%%bash 
for pdf in combined_analysis/*stbl_GENESET/*.txt.summary.pdf; do 
    png=${pdf/.pdf/.png}
    di=`dirname $pdf`
    out=`basename $di`
    outpng=${out/_GENESET/.png}
    outpdf=${out/_GENESET/.pdf}
    
    bash /rumi/shams/abe/Workflows/my_scripts/pdf2png.sh $pdf
    cp -v $pdf combined_analysis/$outpdf
    mv -v $png combined_analysis/$outpng
done 

combined_analysis/hyper_mtyl_120h_delta_stbl_GENESET/120h_delta_stbl.txt.summary.pdf > combined_analysis/hyper_mtyl_120h_delta_stbl_GENESET/120h_delta_stbl.txt.summary.png
done!
'combined_analysis/hyper_mtyl_120h_delta_stbl_GENESET/120h_delta_stbl.txt.summary.pdf' -> 'combined_analysis/hyper_mtyl_120h_delta_stbl.pdf'
renamed 'combined_analysis/hyper_mtyl_120h_delta_stbl_GENESET/120h_delta_stbl.txt.summary.png' -> 'combined_analysis/hyper_mtyl_120h_delta_stbl.png'
combined_analysis/hyper_mtyl_6h_delta_stbl_GENESET/6h_delta_stbl.txt.summary.pdf > combined_analysis/hyper_mtyl_6h_delta_stbl_GENESET/6h_delta_stbl.txt.summary.png
done!
'combined_analysis/hyper_mtyl_6h_delta_stbl_GENESET/6h_delta_stbl.txt.summary.pdf' -> 'combined_analysis/hyper_mtyl_6h_delta_stbl.pdf'
renamed 'combined_analysis/hyper_mtyl_6h_delta_stbl_GENESET/6h_delta_stbl.txt.summary.png' -> 'combined_analysis/hyper_mtyl_6h_delta_stbl.png'
combined_analysis/hyper_mtyl_kg1_delta_stbl_GENESET/kg1_delta_exp.txt.summary.pdf > 

# Ribo-seq 

We have used [Ribolog](https://github.com/goodarzilab/Ribolog) to estimate differential translational efficiency (lnTE). 

In [20]:
%%bash 
declare -a Genesets=('hyper_mtyl' 'hypo_mtyl')
for geneset in "${Genesets[@]}"; do
    echo $time $geneset    
    perl $TEISERDIR/Scripts/teiser_draw_matrix.pl \
    --pvmatrixfile=combined_analysis/${geneset}_lnTE_T_vs_U_GENESET/lnTE_T_vs_U.txt.matrix \
    --summaryfile=combined_analysis/${geneset}_lnTE_T_vs_U_GENESET/lnTE_T_vs_U.txt.summary \
    --expfile=combined_analysis/${geneset}_lnTE_T_vs_U_GENESET/lnTE_T_vs_U.txt \
    --quantized=0 \
    --colmap=$TEISERDIR/Scripts/HEATMAPS/cmap_1.txt \
    --order=0 \
    --min=-3 --max=3 \
    --cluster=5 
done

hyper_mtyl
Reading MI data ... Done.
Start drawing
0.76	-0.4
0
Outputing EPS file combined_analysis/hyper_mtyl_lnTE_T_vs_U_GENESET/lnTE_T_vs_U.txt.summary.eps
Convert to PDF combined_analysis/hyper_mtyl_lnTE_T_vs_U_GENESET/lnTE_T_vs_U.txt.summary.pdf
ps2pdf -dEPSCrop -dAutoRotatePages=/None combined_analysis/hyper_mtyl_lnTE_T_vs_U_GENESET/lnTE_T_vs_U.txt.summary.eps combined_analysis/hyper_mtyl_lnTE_T_vs_U_GENESET/lnTE_T_vs_U.txt.summary.pdf
Finished.
hypo_mtyl
Reading MI data ... Done.
Start drawing
0.76	-0.4
0
Outputing EPS file combined_analysis/hypo_mtyl_lnTE_T_vs_U_GENESET/lnTE_T_vs_U.txt.summary.eps
Convert to PDF combined_analysis/hypo_mtyl_lnTE_T_vs_U_GENESET/lnTE_T_vs_U.txt.summary.pdf
ps2pdf -dEPSCrop -dAutoRotatePages=/None combined_analysis/hypo_mtyl_lnTE_T_vs_U_GENESET/lnTE_T_vs_U.txt.summary.eps combined_analysis/hypo_mtyl_lnTE_T_vs_U_GENESET/lnTE_T_vs_U.txt.summary.pdf
Finished.


In [24]:
%%bash 
for pdf in combined_analysis/*lnTE_T_vs_U_GENESET/*.txt.summary.pdf; do 
    png=${pdf/.pdf/.png}
    di=`dirname $pdf`
    out=`basename $di`
    outpng=${out/_GENESET/.png}
    outpdf=${out/_GENESET/.pdf}
    
    bash /rumi/shams/abe/Workflows/my_scripts/pdf2png.sh $pdf
    cp -v $pdf combined_analysis/$outpdf
    mv -v $png combined_analysis/$outpng
done 

combined_analysis/hyper_mtyl_lnTE_T_vs_U_GENESET/lnTE_T_vs_U.txt.summary.pdf > combined_analysis/hyper_mtyl_lnTE_T_vs_U_GENESET/lnTE_T_vs_U.txt.summary.png
done!
'combined_analysis/hyper_mtyl_lnTE_T_vs_U_GENESET/lnTE_T_vs_U.txt.summary.pdf' -> 'combined_analysis/hyper_mtyl_lnTE_T_vs_U.pdf'
renamed 'combined_analysis/hyper_mtyl_lnTE_T_vs_U_GENESET/lnTE_T_vs_U.txt.summary.png' -> 'combined_analysis/hyper_mtyl_lnTE_T_vs_U.png'
combined_analysis/hypo_mtyl_lnTE_T_vs_U_GENESET/lnTE_T_vs_U.txt.summary.pdf > combined_analysis/hypo_mtyl_lnTE_T_vs_U_GENESET/lnTE_T_vs_U.txt.summary.png
done!
'combined_analysis/hypo_mtyl_lnTE_T_vs_U_GENESET/lnTE_T_vs_U.txt.summary.pdf' -> 'combined_analysis/hypo_mtyl_lnTE_T_vs_U.pdf'
renamed 'combined_analysis/hypo_mtyl_lnTE_T_vs_U_GENESET/lnTE_T_vs_U.txt.summary.png' -> 'combined_analysis/hypo_mtyl_lnTE_T_vs_U.png'
